In [1]:
! easy_install beautifulsoup4
! easy_install  lxml
! easy_install html5lib
! easy_install requests
from bs4 import BeautifulSoup
import requests
import csv
import pandas as pd
import numpy as np


Searching for beautifulsoup4
Best match: beautifulsoup4 4.6.0
Adding beautifulsoup4 4.6.0 to easy-install.pth file

Using /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages
Processing dependencies for beautifulsoup4
Finished processing dependencies for beautifulsoup4
Searching for lxml
Best match: lxml 4.1.0
Adding lxml 4.1.0 to easy-install.pth file

Using /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages
Processing dependencies for lxml
Finished processing dependencies for lxml
Searching for html5lib
Best match: html5lib 0.999999999
Adding html5lib 0.999999999 to easy-install.pth file

Using /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages
Processing dependencies for html5lib
Finished processing dependencies for html5lib
Searching for requests
Best match: requests 2.18.4
Adding requests 2.18.4 to easy-install.pth file

Using /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages
Processing dependencies for requests
Finished processing dependencies for requests


2-Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown below:

In [2]:

source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

soup = BeautifulSoup(source, 'lxml')
table = soup.table
df = pd.read_html(str(table),header=0)
df=df[0]
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [3]:
df=df[df.Borough  != 'Not assigned']

df.head(11)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

In [4]:
df['Neighbourhood'] = np.where(df['Neighbourhood'] == 'Not assigned', df['Borough'] , df['Neighbourhood'])
df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [5]:
 dfnew=df.groupby('Postcode')['Neighbourhood'].agg([('Neighbourhood', ', '.join)]).reset_index()
dfnew

,Postcode,Neighbourhood
0,M1B,"Rouge, Malvern"
1,M1C,"Highland Creek, Rouge Hill, Port Union"
2,M1E,"Guildwood, Morningside, West Hill"
3,M1G,Woburn
4,M1H,Cedarbrae
5,M1J,Scarborough Village
6,M1K,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,"Clairlea, Golden Mile, Oakridge"
8,M1M,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,"Birch Cliff, Cliffside West"


In [6]:
 dfnew.shape 

(103, 2)

In [11]:
geo= pd.read_csv("https://cocl.us/Geospatial_data")
geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.

In an older version of this course, we were leveraging the Google Maps Geocoding API to get the latitude and the longitude coordinates of each neighborhood. However, recently Google started charging for their API: http://geoawesomeness.com/developers-up-in-arms-over-google-maps-api-insane-price-hike/, so we will use the Geocoder Python package instead: https://geocoder.readthedocs.io/index.html.

The problem with this Package is you have to be persistent sometimes in order to get the geographical coordinates of a given postal code. So you can make a call to get the latitude and longitude coordinates of a given postal code and the result would be None, and then make the call again and you would get the coordinates. So, in order to make sure that you get the coordinates for all of our neighborhoods, you can run a while loop for each postal code. Taking postal code M5G as an example, your code would look something like this:


Given that this package can be very unreliable, in case you are not able to get the geographical coordinates of the neighborhoods using the Geocoder package, here is a link to a csv file that has the geographical coordinates of each postal code: http://cocl.us/Geospatial_data

Use the Geocoder package or the csv file to create the following dataframe:

In [13]:
dfx=dfnew.merge(geo,left_on='Postcode', right_on='Postal Code')
dfx.head()

,Postcode,Neighbourhood,Postal Code,Latitude,Longitude
0,M1B,"Rouge, Malvern",M1B,43.806686,-79.194353
1,M1C,"Highland Creek, Rouge Hill, Port Union",M1C,43.784535,-79.160497
2,M1E,"Guildwood, Morningside, West Hill",M1E,43.763573,-79.188711
3,M1G,Woburn,M1G,43.770992,-79.216917
4,M1H,Cedarbrae,M1H,43.773136,-79.239476
